In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib

In [ ]:
# CSV 파일 불러오기
data = pd.read_csv('/content/drive/MyDrive/2023 AI/Final Project/Final_Train/FINAL_TRAIN_V4.csv')

In [ ]:
# 학습에 사용할 특성 선택 (나이, 유전체 정보)
features = ['AGE', 'SEX', 'CENTER', 'ABL1', 'ABL2', 'HRAS', 'NRAS', 'KRAS','CDH10', 'CDH11', 'EGFR', 'HERC2', 'ALK', 'CTNNB1', 'BRAF', 'CDKN2A', 'MITF', 'APC', 'IDH1', 'ATRX']

# 암 종류를 예측할 타겟 선택
target = 'CANCER_TYPE_ACRONYM'

# 특성과 타겟 데이터 분할
X = data[features]
y = data[target]

In [ ]:
preprocessor = ColumnTransformer([
    ('num_imputer', SimpleImputer(strategy='mean'), ['AGE', 'ABL1', 'ABL2', 'HRAS', 'NRAS', 'KRAS','CDH10', 'CDH11', 'EGFR', 'HERC2', 'ALK', 'CTNNB1', 'BRAF', 'CDKN2A', 'MITF', 'APC', 'IDH1', 'ATRX']),
    ('cat_imputer', SimpleImputer(strategy='most_frequent'), ['SEX']),
])

# SEX 열을 숫자로 변환하기 위한 LabelEncoder 생성
sex_encoder = LabelEncoder()

# SEX 열 변환을 위한 라벨 인코딩
X['SEX'] = sex_encoder.fit_transform(X['SEX'])

# 머신 러닝 파이프라인 생성
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler(with_mean=False)),
    ('classifier', RandomForestClassifier()),
])


<ipython-input-31-e35d2209dad5>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['SEX'] = sex_encoder.fit_transform(X['SEX'])


In [ ]:
# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

# 모델 학습
pipeline.fit(X_train, y_train)

# 학습된 모델로 테스트 데이터 예측
predictions = pipeline.predict(X_test)

# 분류 보고서 출력
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

        COAD       1.00      0.74      0.85        19
         LGG       0.83      0.76      0.79        25
        LIHC       0.94      0.88      0.91        17
        LUAD       0.62      0.93      0.75        30
        SKCM       0.74      0.64      0.68        22
        STAD       0.79      0.61      0.69        18

    accuracy                           0.77       131
   macro avg       0.82      0.76      0.78       131
weighted avg       0.80      0.77      0.77       131



In [ ]:
# 학습된 모델 저장
joblib.dump(pipeline, 'cancer_prediction_model.joblib')

# 새로운 데이터 파일 불러오기
new_data = pd.read_csv('/content/drive/MyDrive/2023 AI/Final Project/Final_Test/FINAL_TEST_V4.csv')


# 예측에 사용할 특성 선택
new_features = ['AGE', 'SEX', 'ABL1', 'ABL2', 'HRAS', 'NRAS', 'KRAS','CDH10', 'CDH11', 'EGFR', 'HERC2', 'ALK', 'CTNNB1', 'BRAF', 'CDKN2A', 'MITF', 'APC', 'IDH1', 'ATRX']
# SEX 열 변환을 위한 라벨 인코딩
new_data['SEX'] = sex_encoder.transform(new_data['SEX'])
# 새로운 데이터 예측
new_predictions = pipeline.predict(new_data[new_features])

# 예측 결과 출력
print(new_predictions)

# 예측 결과를 데이터프레임으로 생성
predictions_df = pd.DataFrame({'PATIENT_ID': new_data['PATIENT_ID'], 'CANCER_TYPE': new_predictions})

# 예측 결과를 CSV 파일로 저장
predictions_df.to_csv('/content/drive/MyDrive/2023 AI/Final Project/RESULT/predicted_results_F1.csv')


['LUAD' 'LUAD' 'LGG' 'LIHC' 'STAD' 'STAD' 'STAD' 'STAD' 'STAD' 'STAD'
 'LUAD' 'LUAD' 'SKCM' 'LGG' 'LUAD' 'COAD' 'LUAD' 'STAD' 'LUAD' 'LUAD'
 'LGG' 'LUAD' 'LUAD' 'LIHC' 'LUAD' 'LIHC' 'LIHC' 'LGG' 'LIHC' 'LIHC'
 'COAD' 'LIHC' 'LUAD' 'STAD' 'LIHC' 'LIHC' 'COAD' 'COAD' 'COAD' 'COAD'
 'LUAD' 'COAD' 'LGG' 'SKCM' 'LUAD' 'COAD' 'LUAD' 'LGG' 'LGG' 'SKCM' 'LGG'
 'LGG' 'LGG' 'LGG' 'LIHC' 'COAD' 'LGG' 'LGG' 'LGG' 'COAD' 'LGG' 'SKCM'
 'SKCM' 'SKCM' 'STAD' 'LGG' 'LUAD' 'LGG' 'SKCM' 'SKCM' 'SKCM' 'LGG' 'SKCM'
 'SKCM']
